In [1]:
%pip install gensim nltk scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/balaji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data = [
    ("I love this product", "Positive"),
    ("This is the best experience ever", "Positive"),
    ("I feel amazing after using this", "Positive"),
    ("I enjoy this app", "Positive"),
    ("This product is excellent", "Positive"),
    ("Absolutely wonderful service", "Positive"),
    ("Great quality and performance", "Positive"),
    ("I am very satisfied", "Positive"),
    ("Highly recommend this product", "Positive"),
    ("It works perfectly", "Positive"),
    ("I hate this product", "Negative"),
    ("This is the worst experience", "Negative"),
    ("I feel terrible using this", "Negative"),
    ("Very disappointed with this purchase", "Negative"),
    ("Worst service I have ever experienced", "Negative"),
    ("I will never buy this again", "Negative"),
    ("Horrible quality and performance", "Negative"),
    ("This app is so frustrating", "Negative"),
    ("I regret buying this", "Negative"),
    ("The product stopped working in a week", "Negative"),
    ("Fantastic customer support!", "Positive"),
    ("This exceeded my expectations", "Positive"),
    ("I use this daily and love it", "Positive"),
    ("Very efficient and reliable", "Positive"),
    ("I had a pleasant experience", "Positive"),
    ("Not worth the money", "Negative"),
    ("The delivery was slow and product was damaged", "Negative"),
    ("It broke after a few uses", "Negative"),
    ("Waste of money", "Negative"),
    ("I had high hopes but it disappointed me", "Negative")
]


In [4]:
from  nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import gensim

sentences , labels = zip(*data)

tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Word2Vec model
# word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, sg=1)

model_path = 'GoogleNews-vectors-negative300.bin'
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
  # convert it into tokens
  stop_words = set(stopwords.words('english'))
  text = text.lower()
  tokens = word_tokenize(text)
  # remove stop words
  filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
  return ' '.join(filtered_tokens)


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Function to compute sentence embeddings by averaging word vectors
# normalising labelled data.
def get_sentence_embedding(sentense_tokens, word2vec_model):
  word_vectors = [word2vec_model.wv[word] for word in sentense_tokens if word in word2vec_model.wv]
  if(len(word_vectors)> 0):
    # Axis 0, 1st Column
    return np.mean(word_vectors, axis=0)
  
  # Return a zero vector if no words are in the model
  return np.zeros(word2vec_model.vector_size)


X = np.array([get_sentence_embedding(sentence, word2vec_model) for sentence in tokenized_sentences])
y = np.array(labels)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# stratify=y ensures that the class distribution is maintained across the train and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# classifier = make_pipeline(StandardScaler(), LogisticRegression())
# C is the inverse of regularization strength; smaller values specify stronger regularization.
classifier = make_pipeline(StandardScaler(), LogisticRegression(C=1))
# Train
classifier.fit(X_train, y_train)

# Test
accuracy = classifier.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.17


In [6]:
new_sentences = ["I love this pen", "This is the worst experience"]
new_tokenized = [word_tokenize(sentence.lower()) for sentence in new_sentences]
new_embeddings = np.array([get_sentence_embedding(tokens, word2vec_model) for tokens in new_tokenized])

predictions = classifier.predict(new_embeddings)
for sentence, prediction in zip(new_sentences, predictions):
    print(f"Sentence: {sentence} -> Sentiment: {prediction}")


Sentence: I love this pen -> Sentiment: Positive
Sentence: This is the worst experience -> Sentiment: Negative
